# Assistant-01<br/>The OpenAI reference demo

Reference:
- https://platform.openai.com/docs/assistants/overview

### Get an OpenAI client

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import common
import time
import asyncio

## Get an OpenAI client
client = common.get_openai_client()

### Create an assistant

In [2]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model=common.gpt_deployment_name
)
assistant_id = assistant.id

### Create an assistant thread

In [3]:
thread = client.beta.threads.create()
thread_id = thread.id
thread_id

'thread_TmzUf5OhtQ0WIjSWfxRePaYz'

### Create a user message in the assistant thread

In [4]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    #content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
    content="What is the 1001st prime number. Can you help me?"
)
message

ThreadMessage(id='msg_hHp1JjNmS8uSffJjvKpE5lIH', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What is the 1001st prime number. Can you help me?'), type='text')], created_at=1705096764, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_TmzUf5OhtQ0WIjSWfxRePaYz')

## Run the assistant in the assistant thread

In [5]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

### Wait until the run is completed

In [6]:
while True:
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
  if run.status == "completed":
    break
  # TODO: Is it best practice to wait till completed?
  time.sleep(1)

### Get the messages for the thread and print them

In [7]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
for msg in messages:
  #print(msg)
  content = msg.content
  for line in content:
    txt = line.text
    # TODO: Is there a better way to do this? Should it be sorted by date?
    print(msg.role,txt.value)
  

assistant The 1001st prime number is 7927.
user What is the 1001st prime number. Can you help me?


### List all running assistants in the model

In [8]:
# List the registered assistants
assistants = client.beta.assistants.list()
count = 0
for assistant in assistants.data:
    print(assistant)
    count += 1
    if count ==2:
        break

# TODO: Do the assistants in the model and the threads, that have are not not disposed
# have a time limit or does there have to be a cleanup?


Assistant(id='asst_P8ba3MRBoOv4NbAr9y4VgvFi', created_at=1705096763, description=None, file_ids=[], instructions='You are a personal math tutor. Write and run code to answer math questions.', metadata={}, model='gpt-4-turbo-test', name='Math Tutor', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])
Assistant(id='asst_WrjOLP4OgeU0WVMcGM424Of3', created_at=1705096596, description=None, file_ids=[], instructions='You are an assistant that can get scrape a web page and download the content.', metadata={}, model='gpt-4-turbo-test', name='Web Parser', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])


### Cleanup: clean the assistant and the thread thread

In [11]:
# TODO: Should the assistant live for the life of the application?
assistant_status = client.beta.assistants.delete(assistant_id=assistant_id)
print(assistant_status.deleted)

# TODO: Should the thread live for the life of the execution?
thread_status = client.beta.threads.delete(thread_id=thread_id)
print(thread_status.deleted)

NotFoundError: Error code: 404 - {'error': {'message': "No assistant found with id 'asst_P8ba3MRBoOv4NbAr9y4VgvFi'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}